<a href="https://colab.research.google.com/github/Parv17k/nyc_collision_data_analysis/blob/main/data_cleaning_coll_dataset1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [160]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [161]:
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz

In [162]:
!wget -O longlat.csv -q https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/download/?format=csv&timezone=America/New_York&lang=en&use_labels_for_header=true&csv_separator=%3B

In [163]:
ls

col.csv                       spark-3.1.1-bin-hadoop2.7/
longlat.csv                   spark-3.1.1-bin-hadoop2.7.tgz
ngrok-stable-linux-amd64.zip  spark-3.1.1-bin-hadoop2.7.tgz.1
sample_data/


In [164]:
!tar xf spark-3.1.1-bin-hadoop2.7.tgz

In [165]:
!pip install -q findspark

In [166]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [167]:
import findspark
findspark.init()

In [168]:
import findspark
findspark.init()

In [169]:
findspark.find()

'/content/spark-3.1.1-bin-hadoop2.7'

In [170]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Collision")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [171]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -y ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-03-31 15:07:57--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.212.138.198, 3.222.240.112, 54.197.133.26, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.212.138.198|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14746350 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  14.06M  52.4MB/s    in 0.3s    

2021-03-31 15:07:57 (52.4 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [14746350/14746350]

UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t 

In [172]:
!wget -q https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv ##Data set download

In [173]:
mv rows.csv col.csv ## Linux cmd to rename.

In [174]:
ls

col.csv                         sample_data/
longlat.csv                     spark-3.1.1-bin-hadoop2.7/
ngrok-stable-linux-amd64.zip    spark-3.1.1-bin-hadoop2.7.tgz
ngrok-stable-linux-amd64.zip.1  spark-3.1.1-bin-hadoop2.7.tgz.1


In [175]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
import pyspark
conf = pyspark.SparkConf()
conf.set("spark.driver.memory","8G");
import numpy as np
sc = SparkContext.getOrCreate(conf=conf)
from pyspark.sql.functions import round, col


In [176]:
spark =SparkSession(sc)

In [177]:

# Reading data from local repo. to pyspark.

col=spark.read.option("header",True).csv("./col.csv",inferSchema=True) # data set 1.
longlat=spark.read.option("delimiter",";").csv("./longlat.csv",inferSchema=True,header=True) #location. [Additional Dataset ]

In [178]:
col.count() - col.distinct().count() # I checked for Duplicate rows in our dataset.

0

In [179]:
len(col.columns) # how many columns do I have.

29

In [180]:
col.printSchema() #Schema of my dataset. Issues with our columns types like NUMBER OF PERSONS INJURED: string and It should need to be Integer.

root
 |-- CRASH DATE: string (nullable = true)
 |-- CRASH TIME: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- ZIP CODE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- ON STREET NAME: string (nullable = true)
 |-- CROSS STREET NAME: string (nullable = true)
 |-- OFF STREET NAME: string (nullable = true)
 |-- NUMBER OF PERSONS INJURED: string (nullable = true)
 |-- NUMBER OF PERSONS KILLED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: integer (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: integer (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: string (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: string (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: integer (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 2: strin

In [181]:
from pyspark.sql.types import IntegerType

col=col.withColumn("NUMBER OF PERSONS INJURED", col["NUMBER OF PERSONS INJURED"].cast(IntegerType()))
col=col.withColumn("NUMBER OF CYCLIST KILLED", col["NUMBER OF CYCLIST KILLED"].cast(IntegerType()))
col=col.withColumn("NUMBER OF MOTORIST INJURED", col["NUMBER OF MOTORIST INJURED"].cast(IntegerType()))

# Correcting the column types to Integer which were previously of String types.

In [182]:
col.printSchema() # checking if my schema is correct now ? Yes It is

root
 |-- CRASH DATE: string (nullable = true)
 |-- CRASH TIME: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- ZIP CODE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- ON STREET NAME: string (nullable = true)
 |-- CROSS STREET NAME: string (nullable = true)
 |-- OFF STREET NAME: string (nullable = true)
 |-- NUMBER OF PERSONS INJURED: integer (nullable = true)
 |-- NUMBER OF PERSONS KILLED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: integer (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: integer (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: integer (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: integer (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: integer (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 2: st

In [183]:
col.select("CRASH DATE").show(10)

+----------+
|CRASH DATE|
+----------+
|10/15/2018|
|11/01/2018|
|10/23/2018|
|11/03/2018|
|10/15/2018|
|11/03/2018|
|11/04/2018|
|10/19/2018|
|10/12/2018|
|10/27/2018|
+----------+
only showing top 10 rows



In [184]:
#dropping columns with collision id = null

In [185]:
col.filter(col.COLLISION_ID.isNull()).drop()

DataFrame[CRASH DATE: string, CRASH TIME: string, BOROUGH: string, ZIP CODE: string, LATITUDE: double, LONGITUDE: double, LOCATION: string, ON STREET NAME: string, CROSS STREET NAME: string, OFF STREET NAME: string, NUMBER OF PERSONS INJURED: int, NUMBER OF PERSONS KILLED: int, NUMBER OF PEDESTRIANS INJURED: int, NUMBER OF PEDESTRIANS KILLED: int, NUMBER OF CYCLIST INJURED: int, NUMBER OF CYCLIST KILLED: int, NUMBER OF MOTORIST INJURED: int, NUMBER OF MOTORIST KILLED: int, CONTRIBUTING FACTOR VEHICLE 1: string, CONTRIBUTING FACTOR VEHICLE 2: string, CONTRIBUTING FACTOR VEHICLE 3: string, CONTRIBUTING FACTOR VEHICLE 4: string, CONTRIBUTING FACTOR VEHICLE 5: string, COLLISION_ID: int, VEHICLE TYPE CODE 1: string, VEHICLE TYPE CODE 2: string, VEHICLE TYPE CODE 3: string, VEHICLE TYPE CODE 4: string, VEHICLE TYPE CODE 5: string]

In [186]:
col.count()

1766160

In [187]:
missingLocation=col.filter(col["BOROUGH"].isNull()).filter(col["ZIP CODE"].isNull())

In [188]:
cannotRetriveLocation = missingLocation.filter(col["LONGITUDE"].isNull())

In [189]:
col.count() ## removed the records for which we cannot retrive location at any cost.

1766160

In [190]:
#Infer the schema, and register the DataFrame as a table.
col.createOrReplaceTempView("col");
cannotRetriveLocation.createOrReplaceTempView("toDelete")

In [191]:
col=spark.sql("select * FROM col WHERE col.COLLISION_ID not in (select COLLISION_ID from toDelete where COLLISION_ID is not null)");


In [192]:
col.count()

1588740

In [193]:
col.filter(col["ZIP CODE"].isNull()).filter(col["BOROUGH"].isNotNull()).count()

211

In [194]:
BOROUGH=list(map(lambda x : x["BOROUGH"],col.select("BOROUGH").distinct().collect()))

In [195]:
BOROUGH

[None, 'QUEENS', 'BROOKLYN', 'BRONX', 'MANHATTAN', 'STATEN ISLAND']

In [196]:
longlat.printSchema() # another data set

root
 |-- zip: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- timezone: integer (nullable = true)
 |-- dst: integer (nullable = true)
 |-- geopoint: string (nullable = true)



In [197]:

col.createOrReplaceTempView("col");
longlat.createOrReplaceTempView("longlat");

In [198]:
missingZipCode=col.filter(col['ZIP CODE'].isNull());

In [199]:
missingZipCode.createOrReplaceTempView("missingZipCode");


In [200]:
col.select("NUMBER OF CYCLIST INJURED").distinct().show(); ## lets check.

+-------------------------+
|NUMBER OF CYCLIST INJURED|
+-------------------------+
|                        1|
|                        3|
|                        4|
|                        2|
|                        0|
+-------------------------+



NUMBER OF CYCLIST INJURED Is alredy cleaned.

In [201]:
col.select("NUMBER OF PERSONS INJURED").distinct().show();

+-------------------------+
|NUMBER OF PERSONS INJURED|
+-------------------------+
|                       31|
|                       27|
|                       12|
|                       22|
|                     null|
|                        1|
|                       13|
|                        6|
|                       16|
|                        3|
|                       20|
|                        5|
|                       19|
|                       15|
|                       43|
|                        9|
|                       17|
|                        4|
|                        8|
|                        7|
+-------------------------+
only showing top 20 rows



NUMBER OF PERSONS INJURED has Null value hence making it to Zero.

In [202]:
col=col.fillna(value=0,subset=["NUMBER OF PERSONS INJURED"]) # clean null values to zero.

In [203]:
#NUMBER OF PERSONS KILLED
print(col.select("NUMBER OF PERSONS KILLED").distinct().show());

+------------------------+
|NUMBER OF PERSONS KILLED|
+------------------------+
|                    null|
|                       1|
|                       3|
|                       5|
|                       4|
|                       8|
|                       2|
|                       0|
+------------------------+

None


NUMBER OF PERSONS KILLED has NULL values : replacign them with 0.


In [204]:
col=col.fillna(value=0,subset=["NUMBER OF PERSONS KILLED"])

In [205]:
print(col.select("NUMBER OF PEDESTRIANS INJURED").distinct().show());

+-----------------------------+
|NUMBER OF PEDESTRIANS INJURED|
+-----------------------------+
|                           27|
|                            1|
|                           13|
|                            6|
|                            3|
|                            5|
|                           15|
|                            9|
|                            4|
|                            7|
|                            2|
|                            0|
+-----------------------------+

None


NUMBER OF PEDESTRIANS INJURED has no null value

In [206]:
#NUMBER OF PEDESTRIANS KILLED
print(col.select("NUMBER OF PEDESTRIANS KILLED").distinct().show());

+----------------------------+
|NUMBER OF PEDESTRIANS KILLED|
+----------------------------+
|                           1|
|                           6|
|                           2|
|                           0|
+----------------------------+

None




```
NUMBER OF PEDESTRIANS KILLED has no null value
```



In [207]:
print(col.select("NUMBER OF CYCLIST INJURED").distinct().show());

+-------------------------+
|NUMBER OF CYCLIST INJURED|
+-------------------------+
|                        1|
|                        3|
|                        4|
|                        2|
|                        0|
+-------------------------+

None


NUMBER OF CYCLIST INJURED has no null value

In [208]:
print(col.select("NUMBER OF CYCLIST KILLED").distinct().show());

+------------------------+
|NUMBER OF CYCLIST KILLED|
+------------------------+
|                       1|
|                       2|
|                       0|
+------------------------+

None


NUMBER OF CYCLIST KILLED has no null value

In [209]:
#NUMBER OF MOTORIST INJURED
print(col.select("NUMBER OF MOTORIST INJURED").distinct().show(100));

+--------------------------+
|NUMBER OF MOTORIST INJURED|
+--------------------------+
|                        31|
|                        12|
|                        22|
|                         1|
|                        13|
|                         6|
|                        16|
|                         3|
|                        20|
|                         5|
|                        19|
|                        15|
|                        43|
|                         9|
|                        17|
|                         4|
|                         8|
|                         7|
|                        10|
|                        24|
|                        21|
|                        11|
|                        14|
|                         2|
|                        30|
|                         0|
|                        18|
+--------------------------+

None


NUMBER OF MOTORIST INJURED has no null value

In [210]:
#NUMBER OF MOTORIST INJURED
print(col.select("NUMBER OF MOTORIST KILLED").distinct().show(100));

+-------------------------+
|NUMBER OF MOTORIST KILLED|
+-------------------------+
|                        1|
|                        3|
|                        5|
|                        4|
|                        2|
|                        0|
+-------------------------+

None


NUMBER OF MOTORIST INJURED has no null value

In [211]:
#CONTRIBUTING FACTOR VEHICLE
print(col.select("CONTRIBUTING FACTOR VEHICLE 1").distinct().show(70));

# has numerical values 1 and 80
# has null value.


+-----------------------------+
|CONTRIBUTING FACTOR VEHICLE 1|
+-----------------------------+
|         Following Too Clo...|
|         Traffic Control D...|
|         Accelerator Defec...|
|         Driverless/Runawa...|
|         Windshield Inadeq...|
|         Using On Board Na...|
|                 Unsafe Speed|
|         Shoulders Defecti...|
|               Tinted Windows|
|            Oversized Vehicle|
|         Passing or Lane U...|
|         Lane Marking Impr...|
|         Other Lighting De...|
|         Aggressive Drivin...|
|                         null|
|              Other Vehicular|
|          Driver Inexperience|
|                      Texting|
|          Tow Hitch Defective|
|                       Illnes|
|              Drugs (illegal)|
|                  Unspecified|
|           Pavement Defective|
|         Prescription Medi...|
|         Cell Phone (hand-...|
|         View Obstructed/L...|
|           Lost Consciousness|
|         Reaction to Other...|
|       

In [212]:
#CONTRIBUTING FACTOR VEHICLE
print(col.select("CONTRIBUTING FACTOR VEHICLE 2").distinct().show(100));

# has numerical values 1 and 80.
# has null value.


+-----------------------------+
|CONTRIBUTING FACTOR VEHICLE 2|
+-----------------------------+
|         Following Too Clo...|
|         Traffic Control D...|
|         Driverless/Runawa...|
|         Accelerator Defec...|
|         Windshield Inadeq...|
|         Using On Board Na...|
|                 Unsafe Speed|
|         Shoulders Defecti...|
|               Tinted Windows|
|            Oversized Vehicle|
|         Passing or Lane U...|
|         Lane Marking Impr...|
|         Other Lighting De...|
|                         null|
|         Aggressive Drivin...|
|              Other Vehicular|
|          Driver Inexperience|
|          Tow Hitch Defective|
|                      Texting|
|                       Illnes|
|              Drugs (illegal)|
|                  Unspecified|
|           Pavement Defective|
|         Prescription Medi...|
|         Cell Phone (hand-...|
|         View Obstructed/L...|
|           Lost Consciousness|
|         Reaction to Other...|
|       

In [213]:
#CONTRIBUTING FACTOR VEHICLE
print(col.select("CONTRIBUTING FACTOR VEHICLE 3").distinct().show(100));

# has numerical values 1 and 80.
# has null value.

+-----------------------------+
|CONTRIBUTING FACTOR VEHICLE 3|
+-----------------------------+
|         Following Too Clo...|
|         Traffic Control D...|
|         Driverless/Runawa...|
|         Accelerator Defec...|
|                 Unsafe Speed|
|               Tinted Windows|
|            Oversized Vehicle|
|         Passing or Lane U...|
|         Lane Marking Impr...|
|                         null|
|         Aggressive Drivin...|
|              Other Vehicular|
|          Driver Inexperience|
|                       Illnes|
|              Drugs (illegal)|
|                  Unspecified|
|           Pavement Defective|
|         Prescription Medi...|
|         View Obstructed/L...|
|           Lost Consciousness|
|         Reaction to Other...|
|         Reaction to Uninv...|
|                  Fell Asleep|
|         Tire Failure/Inad...|
|         Outside Car Distr...|
|              Drugs (Illegal)|
|              Fatigued/Drowsy|
|         Driver Inattentio...|
|       

In [214]:
#CONTRIBUTING FACTOR VEHICLE
print(col.select("CONTRIBUTING FACTOR VEHICLE 4").distinct().show(100));
# has null value.

+-----------------------------+
|CONTRIBUTING FACTOR VEHICLE 4|
+-----------------------------+
|         Following Too Clo...|
|         Traffic Control D...|
|         Accelerator Defec...|
|         Driverless/Runawa...|
|         Windshield Inadeq...|
|                 Unsafe Speed|
|         Passing or Lane U...|
|                         null|
|         Aggressive Drivin...|
|              Other Vehicular|
|          Driver Inexperience|
|              Drugs (illegal)|
|                  Unspecified|
|           Pavement Defective|
|         Prescription Medi...|
|         View Obstructed/L...|
|           Lost Consciousness|
|         Reaction to Uninv...|
|                  Fell Asleep|
|         Outside Car Distr...|
|              Drugs (Illegal)|
|              Fatigued/Drowsy|
|         Driver Inattentio...|
|         Failure to Keep R...|
|           Obstruction/Debris|
|         Unsafe Lane Changing|
|          Alcohol Involvement|
|                      Illness|
|       

In [215]:
print(col.select("CONTRIBUTING FACTOR VEHICLE 5").distinct().show(30));
# has null values

+-----------------------------+
|CONTRIBUTING FACTOR VEHICLE 5|
+-----------------------------+
|         Following Too Clo...|
|         Traffic Control D...|
|                 Unsafe Speed|
|         Passing or Lane U...|
|                         null|
|         Aggressive Drivin...|
|              Other Vehicular|
|          Driver Inexperience|
|              Drugs (illegal)|
|                  Unspecified|
|           Pavement Defective|
|         Reaction to Uninv...|
|                  Fell Asleep|
|         Tire Failure/Inad...|
|         Outside Car Distr...|
|              Fatigued/Drowsy|
|         Driver Inattentio...|
|         Failure to Keep R...|
|           Obstruction/Debris|
|          Alcohol Involvement|
|                      Illness|
|         Traffic Control D...|
|             Brakes Defective|
|             Backing Unsafely|
|         Failure to Yield ...|
|             Steering Failure|
|                        Glare|
|          Passing Too Closely|
|       

In [216]:
col.count()

1588740

In [217]:
## Setting all null values to Unknown 
col=col.fillna(value="Unknown",subset=["CONTRIBUTING FACTOR VEHICLE 1","CONTRIBUTING FACTOR VEHICLE 2","CONTRIBUTING FACTOR VEHICLE 3","CONTRIBUTING FACTOR VEHICLE 4","CONTRIBUTING FACTOR VEHICLE 5"])

In [218]:
## Checking what values in VEHICLE TYPE CODE need to be cleaned
print(col.select("VEHICLE TYPE CODE 1","VEHICLE TYPE CODE 2","VEHICLE TYPE CODE 3","VEHICLE TYPE CODE 4","VEHICLE TYPE CODE 5").distinct().show(30));


+--------------------+--------------------+--------------------+--------------------+--------------------+
| VEHICLE TYPE CODE 1| VEHICLE TYPE CODE 2| VEHICLE TYPE CODE 3| VEHICLE TYPE CODE 4| VEHICLE TYPE CODE 5|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|            Flat Bed|                 Bus|                null|                null|                null|
|       Pick-up Truck|       Pick-up Truck|               Sedan|                null|                null|
|         Chassis Cab|Station Wagon/Spo...|               Sedan|                null|                null|
|               Sedan|         Convertible|         Convertible|                null|                null|
|               Sedan|               MOPED|                null|                null|                null|
|             UNKNOWN|             UNKNOWN|                null|                null|                null|
|               trail|               

In [219]:
## Setting null values for vehicle type to unknown
col=col.fillna(value="Unknown",subset=["VEHICLE TYPE CODE 1","VEHICLE TYPE CODE 2","VEHICLE TYPE CODE 3","VEHICLE TYPE CODE 4","VEHICLE TYPE CODE 5"])

In [220]:
## Checking if the values were correctly changed
print(col.select("VEHICLE TYPE CODE 1","VEHICLE TYPE CODE 2","VEHICLE TYPE CODE 3","VEHICLE TYPE CODE 4","VEHICLE TYPE CODE 5").distinct().show(30));

+--------------------+--------------------+--------------------+--------------------+--------------------+
| VEHICLE TYPE CODE 1| VEHICLE TYPE CODE 2| VEHICLE TYPE CODE 3| VEHICLE TYPE CODE 4| VEHICLE TYPE CODE 5|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|               Sedan|               utili|             Unknown|             Unknown|             Unknown|
|                 van|             Unknown|             Unknown|             Unknown|             Unknown|
|                Taxi|       Pick-up Truck|                Taxi|             Unknown|             Unknown|
|                 van|Station Wagon/Spo...|             Unknown|             Unknown|             Unknown|
|               Sedan|Multi-Wheeled Veh...|             Unknown|             Unknown|             Unknown|
|                DELV|               Sedan|             Unknown|             Unknown|             Unknown|
|           Box Truck|Station Wagon/S

In [221]:
### Set all empty values in CROSS STREET NAME to No Value
col=col.fillna(value="NOT PROVIDED",subset=["CROSS STREET NAME"])

In [222]:
### Set all empty values in OFF STREET NAME to No Value
col=col.fillna(value="NOT PROVIDED",subset=["OFF STREET NAME"])

In [223]:
### Set all empty values in ON STREET NAME to No Value
col=col.fillna(value="NOT PROVIDED",subset=["ON STREET NAME"])

In [224]:
## Checking if results have been successfully changed
print(col.select("CROSS STREET NAME","OFF STREET NAME","ON STREET NAME").distinct().show(30));

+--------------------+--------------------+--------------------+
|   CROSS STREET NAME|     OFF STREET NAME|      ON STREET NAME|
+--------------------+--------------------+--------------------+
| NASHVILLE BOULEVARD|        NOT PROVIDED|120 AVENUE       ...|
|        NOT PROVIDED|        NOT PROVIDED|11 STREET        ...|
|         YORK AVENUE|        NOT PROVIDED|EAST 63 STREET   ...|
|        LENOX AVENUE|        NOT PROVIDED|WEST 132 STREET  ...|
|        NOT PROVIDED|34-20     24 STRE...|        NOT PROVIDED|
|        NOT PROVIDED|319       CANAL S...|        NOT PROVIDED|
|             62 ROAD|        NOT PROVIDED|WOODHAVEN BOULEVA...|
|        NOT PROVIDED|88-11     165 STR...|        NOT PROVIDED|
|        NOT PROVIDED|1713      HOBART ...|        NOT PROVIDED|
|        NOT PROVIDED|1631      SHEEPSH...|        NOT PROVIDED|
|        NOT PROVIDED|101       PROSPEC...|        NOT PROVIDED|
|  NORTHERN BOULEVARD|        NOT PROVIDED|248 STREET       ...|
|        NOT PROVIDED|135

In [225]:
#Check CRASH DATE format
col.select("CRASH DATE").show(10)

+----------+
|CRASH DATE|
+----------+
|10/15/2018|
|11/01/2018|
|10/23/2018|
|11/03/2018|
|10/15/2018|
|11/03/2018|
|11/04/2018|
|10/19/2018|
|10/12/2018|
|10/27/2018|
+----------+
only showing top 10 rows



In [226]:
#Change CRASH DATE type from String to Date
from pyspark.sql.types import DateType
from pyspark.sql.functions import to_date

#col.withColumn("CRASH DATE", col['CRASH DATE'].cast(DateType())).show()

 
col= col.withColumn('CRASH DATE',to_date(col["CRASH DATE"], 'MM/dd/yyyy'))
col.printSchema()

root
 |-- CRASH DATE: date (nullable = true)
 |-- CRASH TIME: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- ZIP CODE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- ON STREET NAME: string (nullable = false)
 |-- CROSS STREET NAME: string (nullable = false)
 |-- OFF STREET NAME: string (nullable = false)
 |-- NUMBER OF PERSONS INJURED: integer (nullable = true)
 |-- NUMBER OF PERSONS KILLED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: integer (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: integer (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: integer (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: integer (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: integer (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = false)
 |-- CONTRIBUTING FACTOR VEHICLE 2: 

In [227]:
# Check if format is correct
col.select("CRASH DATE").show(10)

+----------+
|CRASH DATE|
+----------+
|2018-10-15|
|2018-11-01|
|2018-10-23|
|2018-11-03|
|2018-10-15|
|2018-11-03|
|2018-11-04|
|2018-10-19|
|2018-10-12|
|2018-10-27|
+----------+
only showing top 10 rows



In [228]:
## Converting CRASH TIME type: String to timestamp 
from pyspark.sql.types import DateType
from pyspark.sql.functions import to_timestamp

col= col.withColumn('CRASH TIME',to_timestamp(col["CRASH TIME"], 'H:mm'))
col.printSchema()

root
 |-- CRASH DATE: date (nullable = true)
 |-- CRASH TIME: timestamp (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- ZIP CODE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- ON STREET NAME: string (nullable = false)
 |-- CROSS STREET NAME: string (nullable = false)
 |-- OFF STREET NAME: string (nullable = false)
 |-- NUMBER OF PERSONS INJURED: integer (nullable = true)
 |-- NUMBER OF PERSONS KILLED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: integer (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: integer (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: integer (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: integer (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: integer (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = false)
 |-- CONTRIBUTING FACTOR VEHICLE 

In [229]:
## Checking if format is correct
col.select("CRASH TIME").show(10)

+-------------------+
|         CRASH TIME|
+-------------------+
|1970-01-01 08:00:00|
|1970-01-01 16:30:00|
|1970-01-01 20:12:00|
|1970-01-01 14:35:00|
|1970-01-01 17:30:00|
|1970-01-01 20:40:00|
|1970-01-01 13:50:00|
|1970-01-01 12:10:00|
|1970-01-01 18:20:00|
|1970-01-01 14:45:00|
+-------------------+
only showing top 10 rows



In [230]:
### Removing YYYY-MM-DD from CRASH TIME
from pyspark.sql.types import DateType
from pyspark.sql.functions import date_format
from pyspark.sql import functions as f

col = col.withColumn("CRASH TIME", f.date_format("CRASH TIME", 'HH:mm:ss'))


In [231]:
## Checking Results of Format
col.select("CRASH TIME").show(10)

+----------+
|CRASH TIME|
+----------+
|  08:00:00|
|  16:30:00|
|  20:12:00|
|  14:35:00|
|  17:30:00|
|  20:40:00|
|  13:50:00|
|  12:10:00|
|  18:20:00|
|  14:45:00|
+----------+
only showing top 10 rows



In [233]:
col.show()

+----------+----------+-------------+--------+---------+----------+--------------------+--------------------+-------------------+--------------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+--------------------+--------------------+-------------------+-------------------+-------------------+
|CRASH DATE|CRASH TIME|      BOROUGH|ZIP CODE| LATITUDE| LONGITUDE|            LOCATION|      ON STREET NAME|  CROSS STREET NAME|     OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLED|CONTRI

# ***TODO***
Treat Null Values for all column to  Unknown. 1,2,3,4,5


Treat **1 and 80** for CONTRIBUTING FACTOR VEHICLE 1,2,3

In [232]:
# zip,borough,location,long,lat, [contributing1,2,3,4], killed and injured [3-4],COLLISION_ID.